In [1]:
import logging
import torch

lower = True
seq_length = 100
report_every = 100000
shuffle = 1

logger = logging.getLogger(__name__)

#pad word i.e. if the max sequence of the sentence is 8, all 
PAD_WORD = '<blank>'

#unknow word
UNK_WORD = '<unk>'

#beginning of the sentence!
BOS_WORD = '<s>'

#end of sentence word, use this to mark the end of the sentence
EOS_WORD = '</s>'


#return a dict of each vocabulary 
def makeVocabulary(filenames, size):
    vocab = Dict([PAD_WORD, UNK_WORD,
                      BOS_WORD, EOS_WORD], lower=lower)
    for filename in filenames:
        with open(filename, encoding='utf-8') as f:
            for sent in f.readlines():
                for word in sent.strip().split(' '):
                    vocab.add(word)

    originalSize = vocab.size()
    #vocab = vocab.prune(size)
    print('Created dictionary of size %d (pruned from %d)' %
                (vocab.size(), originalSize))

    return vocab


def initVocabulary(name, dataFiles, vocabFile, vocabSize):
    vocab = None
    if vocabFile is not None:
        # If given, load existing word dictionary.
        logger.info('Reading ' + name + ' vocabulary from \'' + vocabFile + '\'...')
        vocab = Dict(lower=lower)
        vocab.loadFile(vocabFile)
        logger.info('Loaded ' + str(vocab.size()) + ' ' + name + ' words')

    if vocab is None:
        # If a dictionary is still missing, generate it.
        logger.info('Building ' + name + ' vocabulary...')
        genWordVocab = makeVocabulary(dataFiles, vocabSize)

        vocab = genWordVocab

    return vocab


def saveVocabulary(name, vocab, file):
    logger.info('Saving ' + name + ' vocabulary to \'' + file + '\'...')
    vocab.writeFile(file)


def makeData(srcFile, bioFile, featFiles, tgtFile, srcDicts, bioDicts, featDicts, tgtDicts):
    src, tgt = [], []
    bio = []
    feats = []
    switch, c_tgt = [], []
    sizes = []
    count, ignored = 0, 0

    logger.info('Processing %s & %s ...' % (srcFile, tgtFile))
    srcF = open(srcFile, encoding='utf-8')
    tgtF = open(tgtFile, encoding='utf-8')
    bioF = open(bioFile, encoding='utf-8')
    featFs = [open(x, encoding='utf-8') for x in featFiles]
    turn = 0
    iter_num = 0
    while iter_num < 72000:
        turn += 1
        print('this turn is', turn)
        #read one line in the file by one time
        sline = srcF.readline()
        tline = tgtF.readline()
        bioLine = bioF.readline()
        featLines = [x.readline() for x in featFs]
        iter_num += 1
        # normal end of file
        if sline == "" and tline == "":
            break

        # source or target does not have same number of lines
        if sline == "" or tline == "":
            logger.info('WARNING: source and target do not have the same number of sentences')
            break

        sline = sline.strip()
        tline = tline.strip()
        bioLine = bioLine.strip()
        featLines = [line.strip() for line in featLines]

        # source and/or target are empty
        if sline == "" or tline == "":
            logger.info('WARNING: ignoring an empty line (' + str(count + 1) + ')')
            continue

        srcWords = sline.split(' ')
        tgtWords = tline.split(' ')
        bioWords = bioLine.split(' ')
        featWords = [x.split(' ') for x in featLines]

        if len(srcWords) <= seq_length and len(tgtWords) <= seq_length:
            src += [srcDicts.convertToIdx(srcWords, UNK_WORD)]
            bio += [bioDicts.convertToIdx(bioWords, UNK_WORD)]
            feats += [[featDicts.convertToIdx(x, UNK_WORD) for x in featWords]]
            tgt += [tgtDicts.convertToIdx(tgtWords,
                                          UNK_WORD,
                                          BOS_WORD,
                                          EOS_WORD)]
            switch_buf = [0] * (len(tgtWords) + 2)
            c_tgt_buf = [0] * (len(tgtWords) + 2)
            for idx, tgt_word in enumerate(tgtWords):
                word_id = tgtDicts.lookup(tgt_word, None)
                if word_id is None:
                    if tgt_word in srcWords:
                        copy_position = srcWords.index(tgt_word)
                        switch_buf[idx + 1] = 1
                        c_tgt_buf[idx + 1] = copy_position
            switch.append(torch.FloatTensor(switch_buf))
            c_tgt.append(torch.LongTensor(c_tgt_buf))

            sizes += [len(srcWords)]
        else:
            ignored += 1

        count += 1

        if count % report_every == 0:
            logger.info('... %d sentences prepared' % count)
    print('iteration has ended')
    srcF.close()
    tgtF.close()
    bioF.close()
    for x in featFs:
        x.close()
    print('all files have been closed')
    if shuffle == 1:
        logger.info('... shuffling sentences')
        perm = torch.randperm(len(src))
        src = [src[idx] for idx in perm]
        tgt = [tgt[idx] for idx in perm]
        bio = [bio[idx] for idx in perm]
        feats = [feats[idx] for idx in perm]
        switch = [switch[idx] for idx in perm]
        c_tgt = [c_tgt[idx] for idx in perm]
        sizes = [sizes[idx] for idx in perm]

    print('... sorting sentences by size')
    _, perm = torch.sort(torch.Tensor(sizes))
    src = [src[idx] for idx in perm]
    tgt = [tgt[idx] for idx in perm]
    bio = [bio[idx] for idx in perm]
    feats = [feats[idx] for idx in perm]
    switch = [switch[idx] for idx in perm]
    c_tgt = [c_tgt[idx] for idx in perm]

    print('Prepared %d sentences (%d ignored due to length == 0 or > %d)' %
                (len(src), ignored, seq_length))
    return src, bio, feats, tgt, switch, c_tgt


def prepare_data_online(train_src, src_vocab, train_bio, bio_vocab, train_feats, feat_vocab, train_tgt, tgt_vocab):
    dicts = {}
    dicts['src'] = initVocabulary('source', [train_src], src_vocab, 0)
    dicts['bio'] = initVocabulary('bio', [train_bio], bio_vocab, 0)
    dicts['feat'] = initVocabulary('feat', train_feats, feat_vocab, 0)
    dicts['tgt'] = initVocabulary('target', [train_tgt], tgt_vocab, 0)

    logger.info('Preparing training ...')
    train = {}
    train['src'], train['bio'], train['feats'], \
    train['tgt'], train['switch'], train['c_tgt'] = makeData(train_src, train_bio, train_feats,
                                                             train_tgt,
                                                             dicts['src'], dicts['bio'], dicts['feat'],
                                                             dicts['tgt'])

    dataset = {'dicts': dicts,
               'train': train,
               # 'valid': valid
               }
    return dataset

In [2]:
import torch


class Dict:
    def __init__(self, data=None, lower=False):
        self.idxToLabel = {}
        self.labelToIdx = {}
        self.frequencies = {}
        self.lower = lower

        # Special entries will not be pruned.
        self.special = []

        if data is not None:
            #if the it's the name of the whole file, then load the file
            if type(data) == str:
                self.loadFile(data)
            else:
                self.addSpecials(data)

    def size(self):
        return len(self.idxToLabel)

    # Load entries from a file.
    def loadFile(self, filename):
        for line in open(filename, encoding='utf-8'):
            fields = line.split(' ')
            label = fields[0]
            idx = int(fields[1])
            self.add(label, idx)

    # Write entries to a file.
    def writeFile(self, filename):
        with open(filename, 'w', encoding='utf-8') as file:
            for i in range(self.size()):
                label = self.idxToLabel[i]
                file.write('%s %d\n' % (label, i))

        file.close()

    def lookup(self, key, default=None):
        key = key.lower() if self.lower else key
        try:
            return self.labelToIdx[key]
        except KeyError:
            return default

    def getLabel(self, idx, default=None):
        try:
            return self.idxToLabel[idx]
        except KeyError:
            return default

    # Mark this `label` and `idx` as special (i.e. will not be pruned).
    def addSpecial(self, label, idx=None):
        idx = self.add(label, idx)
        self.special += [idx]

    # Mark all labels in `labels` as specials (i.e. will not be pruned).
    def addSpecials(self, labels):
        for label in labels:
            self.addSpecial(label)

    # Add `label` in the dictionary. Use `idx` as its index if given.
    def add(self, label, idx=None):
        label = label.lower() if self.lower else label
        if idx is not None:
            self.idxToLabel[idx] = label
            self.labelToIdx[label] = idx
        else:
            if label in self.labelToIdx:
                idx = self.labelToIdx[label]
            else:
                idx = len(self.idxToLabel)
                self.idxToLabel[idx] = label
                self.labelToIdx[label] = idx

        if idx not in self.frequencies:
            self.frequencies[idx] = 1
        else:
            self.frequencies[idx] += 1

        return idx

    # Return a new dictionary with the `size` most frequent entries.
    def prune(self, size):
        if size >= self.size():
            return self

        # Only keep the `size` most frequent entries.
        freq = torch.Tensor(
                [self.frequencies[i] for i in range(len(self.frequencies))])
        _, idx = torch.sort(freq, 0, True)

        newDict = Dict()
        newDict.lower = self.lower

        # Add special entries in all cases.
        for i in self.special:
            newDict.addSpecial(self.idxToLabel[i])

        for i in idx[:size]:
            newDict.add(self.idxToLabel[i])

        return newDict

    # Convert `labels` to indices. Use `unkWord` if not found.
    # Optionally insert `bosWord` at the beginning and `eosWord` at the .
    def convertToIdx(self, labels, unkWord, bosWord=None, eosWord=None):
        vec = []

        if bosWord is not None:
            vec += [self.lookup(bosWord)]

        unk = self.lookup(unkWord)
        vec += [self.lookup(label, default=unk) for label in labels]

        if eosWord is not None:
            vec += [self.lookup(eosWord)]

        return torch.LongTensor(vec)

    # Convert `idx` to labels. If index `stop` is reached, convert it and return.
    def convertToLabels(self, idx, stop):
        labels = []

        for i in idx:
            labels += [self.getLabel(i)]
            if i == stop:
                break

        return labels



In [ ]:
#make vocab for scr, bio, fea, tgt
src_vocab = makeVocabulary(['redistribute/QG/train/train.txt.source.txt'],0)
bio_vocab = makeVocabulary(['redistribute/QG/train/train.txt.bio'],0)
feat_vocab = makeVocabulary(['redistribute/QG/train/train.txt.ner', 'redistribute/QG/train/train.txt.case','redistribute/QG/train/train.txt.pos'],0)
tgt_vocab = makeVocabulary(['redistribute/QG/train/train.txt.target.txt'],0)

Created dictionary of size 78330 (pruned from 78330)
Created dictionary of size 7 (pruned from 7)
Created dictionary of size 63 (pruned from 63)
Created dictionary of size 39163 (pruned from 39163)


In [11]:
src_vocab.writeFile('src_vocab')
bio_vocab.writeFile('bio_vocab')
feat_vocab.writeFile('feat_vocab')
tgt_vocab.writeFile('tgt_vocab')

In [3]:
src_file = 'redistribute/QG/train/train.txt.source.txt'
bio_file = 'redistribute/QG/train/train.txt.bio'
feat_file = ['redistribute/QG/train/train.txt.ner', 'redistribute/QG/train/train.txt.case','redistribute/QG/train/train.txt.pos']
tgt_file = 'redistribute/QG/train/train.txt.target.txt'

dataset = prepare_data_online(src_file, None, bio_file, None, feat_file, None,tgt_file, None)

Created dictionary of size 78330 (pruned from 78330)
Created dictionary of size 7 (pruned from 7)
Created dictionary of size 63 (pruned from 63)
Created dictionary of size 39163 (pruned from 39163)
this turn is 1
this turn is 2
this turn is 3
this turn is 4
this turn is 5
this turn is 6
this turn is 7
this turn is 8
this turn is 9
this turn is 10
this turn is 11
this turn is 12
this turn is 13
this turn is 14
this turn is 15
this turn is 16
this turn is 17
this turn is 18
this turn is 19
this turn is 20
this turn is 21
this turn is 22
this turn is 23
this turn is 24
this turn is 25
this turn is 26
this turn is 27
this turn is 28
this turn is 29
this turn is 30
this turn is 31
this turn is 32
this turn is 33
this turn is 34
this turn is 35
this turn is 36
this turn is 37
this turn is 38
this turn is 39
this turn is 40
this turn is 41
this turn is 42
this turn is 43
this turn is 44
this turn is 45
this turn is 46
this turn is 47
this turn is 48
this turn is 49
this turn is 50
this turn i

this turn is 933
this turn is 934
this turn is 935
this turn is 936
this turn is 937
this turn is 938
this turn is 939
this turn is 940
this turn is 941
this turn is 942
this turn is 943
this turn is 944
this turn is 945
this turn is 946
this turn is 947
this turn is 948
this turn is 949
this turn is 950
this turn is 951
this turn is 952
this turn is 953
this turn is 954
this turn is 955
this turn is 956
this turn is 957
this turn is 958
this turn is 959
this turn is 960
this turn is 961
this turn is 962
this turn is 963
this turn is 964
this turn is 965
this turn is 966
this turn is 967
this turn is 968
this turn is 969
this turn is 970
this turn is 971
this turn is 972
this turn is 973
this turn is 974
this turn is 975
this turn is 976
this turn is 977
this turn is 978
this turn is 979
this turn is 980
this turn is 981
this turn is 982
this turn is 983
this turn is 984
this turn is 985
this turn is 986
this turn is 987
this turn is 988
this turn is 989
this turn is 990
this turn is 9

this turn is 1396
this turn is 1397
this turn is 1398
this turn is 1399
this turn is 1400
this turn is 1401
this turn is 1402
this turn is 1403
this turn is 1404
this turn is 1405
this turn is 1406
this turn is 1407
this turn is 1408
this turn is 1409
this turn is 1410
this turn is 1411
this turn is 1412
this turn is 1413
this turn is 1414
this turn is 1415
this turn is 1416
this turn is 1417
this turn is 1418
this turn is 1419
this turn is 1420
this turn is 1421
this turn is 1422
this turn is 1423
this turn is 1424
this turn is 1425
this turn is 1426
this turn is 1427
this turn is 1428
this turn is 1429
this turn is 1430
this turn is 1431
this turn is 1432
this turn is 1433
this turn is 1434
this turn is 1435
this turn is 1436
this turn is 1437
this turn is 1438
this turn is 1439
this turn is 1440
this turn is 1441
this turn is 1442
this turn is 1443
this turn is 1444
this turn is 1445
this turn is 1446
this turn is 1447
this turn is 1448
this turn is 1449
this turn is 1450
this turn 

this turn is 1858
this turn is 1859
this turn is 1860
this turn is 1861
this turn is 1862
this turn is 1863
this turn is 1864
this turn is 1865
this turn is 1866
this turn is 1867
this turn is 1868
this turn is 1869
this turn is 1870
this turn is 1871
this turn is 1872
this turn is 1873
this turn is 1874
this turn is 1875
this turn is 1876
this turn is 1877
this turn is 1878
this turn is 1879
this turn is 1880
this turn is 1881
this turn is 1882
this turn is 1883
this turn is 1884
this turn is 1885
this turn is 1886
this turn is 1887
this turn is 1888
this turn is 1889
this turn is 1890
this turn is 1891
this turn is 1892
this turn is 1893
this turn is 1894
this turn is 1895
this turn is 1896
this turn is 1897
this turn is 1898
this turn is 1899
this turn is 1900
this turn is 1901
this turn is 1902
this turn is 1903
this turn is 1904
this turn is 1905
this turn is 1906
this turn is 1907
this turn is 1908
this turn is 1909
this turn is 1910
this turn is 1911
this turn is 1912
this turn 

this turn is 2319
this turn is 2320
this turn is 2321
this turn is 2322
this turn is 2323
this turn is 2324
this turn is 2325
this turn is 2326
this turn is 2327
this turn is 2328
this turn is 2329
this turn is 2330
this turn is 2331
this turn is 2332
this turn is 2333
this turn is 2334
this turn is 2335
this turn is 2336
this turn is 2337
this turn is 2338
this turn is 2339
this turn is 2340
this turn is 2341
this turn is 2342
this turn is 2343
this turn is 2344
this turn is 2345
this turn is 2346
this turn is 2347
this turn is 2348
this turn is 2349
this turn is 2350
this turn is 2351
this turn is 2352
this turn is 2353
this turn is 2354
this turn is 2355
this turn is 2356
this turn is 2357
this turn is 2358
this turn is 2359
this turn is 2360
this turn is 2361
this turn is 2362
this turn is 2363
this turn is 2364
this turn is 2365
this turn is 2366
this turn is 2367
this turn is 2368
this turn is 2369
this turn is 2370
this turn is 2371
this turn is 2372
this turn is 2373
this turn 

...

this turn is 70130
this turn is 70131
this turn is 70132
this turn is 70133
this turn is 70134
this turn is 70135
this turn is 70136
this turn is 70137
this turn is 70138
this turn is 70139
this turn is 70140
this turn is 70141
this turn is 70142
this turn is 70143
this turn is 70144
this turn is 70145
this turn is 70146
this turn is 70147
this turn is 70148
this turn is 70149
this turn is 70150
this turn is 70151
this turn is 70152
this turn is 70153
this turn is 70154
this turn is 70155
this turn is 70156
this turn is 70157
this turn is 70158
this turn is 70159
this turn is 70160
this turn is 70161
this turn is 70162
this turn is 70163
this turn is 70164
this turn is 70165
this turn is 70166
this turn is 70167
this turn is 70168
this turn is 70169
this turn is 70170
this turn is 70171
this turn is 70172
this turn is 70173
this turn is 70174
this turn is 70175
this turn is 70176
this turn is 70177
this turn is 70178
this turn is 70179
this turn is 70180
this turn is 70181
this turn is

this turn is 70571
this turn is 70572
this turn is 70573
this turn is 70574
this turn is 70575
this turn is 70576
this turn is 70577
this turn is 70578
this turn is 70579
this turn is 70580
this turn is 70581
this turn is 70582
this turn is 70583
this turn is 70584
this turn is 70585
this turn is 70586
this turn is 70587
this turn is 70588
this turn is 70589
this turn is 70590
this turn is 70591
this turn is 70592
this turn is 70593
this turn is 70594
this turn is 70595
this turn is 70596
this turn is 70597
this turn is 70598
this turn is 70599
this turn is 70600
this turn is 70601
this turn is 70602
this turn is 70603
this turn is 70604
this turn is 70605
this turn is 70606
this turn is 70607
this turn is 70608
this turn is 70609
this turn is 70610
this turn is 70611
this turn is 70612
this turn is 70613
this turn is 70614
this turn is 70615
this turn is 70616
this turn is 70617
this turn is 70618
this turn is 70619
this turn is 70620
this turn is 70621
this turn is 70622
this turn is

this turn is 71046
this turn is 71047
this turn is 71048
this turn is 71049
this turn is 71050
this turn is 71051
this turn is 71052
this turn is 71053
this turn is 71054
this turn is 71055
this turn is 71056
this turn is 71057
this turn is 71058
this turn is 71059
this turn is 71060
this turn is 71061
this turn is 71062
this turn is 71063
this turn is 71064
this turn is 71065
this turn is 71066
this turn is 71067
this turn is 71068
this turn is 71069
this turn is 71070
this turn is 71071
this turn is 71072
this turn is 71073
this turn is 71074
this turn is 71075
this turn is 71076
this turn is 71077
this turn is 71078
this turn is 71079
this turn is 71080
this turn is 71081
this turn is 71082
this turn is 71083
this turn is 71084
this turn is 71085
this turn is 71086
this turn is 71087
this turn is 71088
this turn is 71089
this turn is 71090
this turn is 71091
this turn is 71092
this turn is 71093
this turn is 71094
this turn is 71095
this turn is 71096
this turn is 71097
this turn is

this turn is 71513
this turn is 71514
this turn is 71515
this turn is 71516
this turn is 71517
this turn is 71518
this turn is 71519
this turn is 71520
this turn is 71521
this turn is 71522
this turn is 71523
this turn is 71524
this turn is 71525
this turn is 71526
this turn is 71527
this turn is 71528
this turn is 71529
this turn is 71530
this turn is 71531
this turn is 71532
this turn is 71533
this turn is 71534
this turn is 71535
this turn is 71536
this turn is 71537
this turn is 71538
this turn is 71539
this turn is 71540
this turn is 71541
this turn is 71542
this turn is 71543
this turn is 71544
this turn is 71545
this turn is 71546
this turn is 71547
this turn is 71548
this turn is 71549
this turn is 71550
this turn is 71551
this turn is 71552
this turn is 71553
this turn is 71554
this turn is 71555
this turn is 71556
this turn is 71557
this turn is 71558
this turn is 71559
this turn is 71560
this turn is 71561
this turn is 71562
this turn is 71563
this turn is 71564
this turn is

this turn is 71980
this turn is 71981
this turn is 71982
this turn is 71983
this turn is 71984
this turn is 71985
this turn is 71986
this turn is 71987
this turn is 71988
this turn is 71989
this turn is 71990
this turn is 71991
this turn is 71992
this turn is 71993
this turn is 71994
this turn is 71995
this turn is 71996
this turn is 71997
this turn is 71998
this turn is 71999
this turn is 72000
iteration has ended
all files have been closed
... sorting sentences by size
Prepared 71526 sentences (474 ignored due to length == 0 or > 100)


In [5]:
dataset.keys()

dict_keys(['dicts', 'train'])

In [9]:
dataset['train']['src']

[tensor([37657,  1483]),
 tensor([ 283, 4104,  214, 4606]),
 tensor([  283, 15563, 52315,    26]),
 tensor([  243,  1660, 37107,    26]),
 tensor([137, 317, 146,  26]),
 tensor([  283, 15563, 52315,    26]),
 tensor([9627,    5, 5346,   26]),
 tensor([14493, 14494,     9, 14169,    26]),
 tensor([ 447, 3936,    5, 9415,   26]),
 tensor([17137,   983,     5, 22019,    26]),
 tensor([45931, 11690,   102, 45943,    26]),
 tensor([    4,   176, 23428,  6305,    26]),
 tensor([67476,  3588,  6159, 67477,    26]),
 tensor([12234,  4858,    24, 12259,    26]),
 tensor([32591,  5595,     9,    71,    26]),
 tensor([ 3047,  4229, 15356,  1800,    26]),
 tensor([    6, 55162,   131,  4579,    26]),
 tensor([  241,   144,  4981, 53844,    26]),
 tensor([  140,  6035,     5, 51450,    26]),
 tensor([16961,   176,   617, 10178,    26]),
 tensor([15845, 10506, 32632,  1813,   632]),
 tensor([8099,  906, 6898, 1848,   26]),
 tensor([  140,  4258,     5, 13911,    26]),
 tensor([   4, 1741,  430, 4640